In [3]:
import numpy as np
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt


In [5]:
df = pd.read_csv('Churn(sql).csv')
df.head()


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [7]:
import sqlite3


In [9]:
# Create an in-memory SQLite database
conn = sqlite3.connect(':memory:')


In [19]:
# Load the DataFrame into the SQLite database, replacing the existing table if it already exists
df.to_sql('churn_data', conn, if_exists='replace', index=False)


10000

In [21]:
# Function to run SQL queries
def run_query(query):
    return pd.read_sql(query, conn)


In [23]:
# Example query to check the first 5 rows
run_query("SELECT * FROM churn_data LIMIT 5")


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [25]:
# View column names and types
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [27]:
df.describe()


,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


In [29]:
churn_rate_query = """
SELECT COUNT(*) * 100.0 / (SELECT COUNT(*) FROM churn_data) AS churn_rate
FROM churn_data
WHERE Exited = 1
"""
churn_rate = run_query(churn_rate_query)
print(churn_rate)


   churn_rate
0       20.37


In [31]:
geography_churn_query = """
SELECT Geography, 
       COUNT(*) AS total_customers, 
       SUM(CASE WHEN Exited = 1 THEN 1 ELSE 0 END) AS churned_customers,
       SUM(CASE WHEN Exited = 1 THEN 1 ELSE 0 END) * 100.0 / COUNT(*) AS churn_rate
FROM churn_data
GROUP BY Geography
"""
geography_churn = run_query(geography_churn_query)
print(geography_churn)


  Geography  total_customers  churned_customers  churn_rate
0    France             5014                810   16.154767
1   Germany             2509                814   32.443204
2     Spain             2477                413   16.673395


In [33]:
age_distribution_query = """
SELECT Age, 
       COUNT(CASE WHEN Exited = 1 THEN 1 ELSE NULL END) AS churned,
       COUNT(CASE WHEN Exited = 0 THEN 1 ELSE NULL END) AS not_churned
FROM churn_data
GROUP BY Age
"""
age_distribution = run_query(age_distribution_query)
print(age_distribution)


    Age  churned  not_churned
0    18        2           20
1    19        1           26
2    20        2           38
3    21        3           50
4    22       12           72
..  ...      ...          ...
65   83        0            1
66   84        1            1
67   85        0            1
68   88        0            1
69   92        0            2

[70 rows x 3 columns]


In [35]:
factors_query = """
SELECT CreditScore, Balance, NumOfProducts, 
       SUM(CASE WHEN Exited = 1 THEN 1 ELSE 0 END) AS churned_customers,
       SUM(CASE WHEN Exited = 1 THEN 1 ELSE 0 END) * 100.0 / COUNT(*) AS churn_rate
FROM churn_data
GROUP BY CreditScore, Balance, NumOfProducts
ORDER BY churn_rate DESC
LIMIT 10
"""
churn_factors = run_query(factors_query)
print(churn_factors)


   CreditScore    Balance  NumOfProducts  churned_customers  churn_rate
0          350       0.00              1                  2       100.0
1          350  109733.20              2                  1       100.0
2          350  111098.85              1                  1       100.0
3          350  152677.48              1                  1       100.0
4          351  163146.46              1                  1       100.0
5          358  143542.36              3                  1       100.0
6          359  128747.69              1                  1       100.0
7          363  146098.43              3                  1       100.0
8          365  127760.07              1                  1       100.0
9          367   93608.28              1                  1       100.0


In [37]:
tenure_query = """
SELECT Tenure, 
       COUNT(*) AS total_customers, 
       SUM(CASE WHEN Exited = 1 THEN 1 ELSE 0 END) AS churned_customers,
       SUM(CASE WHEN Exited = 1 THEN 1 ELSE 0 END) * 100.0 / COUNT(*) AS churn_rate
FROM churn_data
GROUP BY Tenure
ORDER BY Tenure
"""
tenure_analysis = run_query(tenure_query)
print(tenure_analysis)


    Tenure  total_customers  churned_customers  churn_rate
0        0              413                 95   23.002421
1        1             1035                232   22.415459
2        2             1048                201   19.179389
3        3             1009                213   21.110010
4        4              989                203   20.525784
5        5             1012                209   20.652174
6        6              967                196   20.268873
7        7             1028                177   17.217899
8        8             1025                197   19.219512
9        9              984                213   21.646341
10      10              490                101   20.612245


In [39]:
credit_score_query = """
SELECT CreditScore, 
       COUNT(*) AS total_customers, 
       SUM(CASE WHEN Exited = 1 THEN 1 ELSE 0 END) AS churned_customers,
       SUM(CASE WHEN Exited = 1 THEN 1 ELSE 0 END) * 100.0 / COUNT(*) AS churn_rate
FROM churn_data
GROUP BY CreditScore
ORDER BY CreditScore
"""
credit_score_analysis = run_query(credit_score_query)
print(credit_score_analysis)


     CreditScore  total_customers  churned_customers  churn_rate
0            350                5                  5  100.000000
1            351                1                  1  100.000000
2            358                1                  1  100.000000
3            359                1                  1  100.000000
4            363                1                  1  100.000000
..           ...              ...                ...         ...
455          846                5                  2   40.000000
456          847                6                  2   33.333333
457          848                5                  0    0.000000
458          849                8                  2   25.000000
459          850              233                 43   18.454936

[460 rows x 4 columns]


In [41]:
balance_query = """
SELECT Balance, 
       COUNT(*) AS total_customers, 
       SUM(CASE WHEN Exited = 1 THEN 1 ELSE 0 END) AS churned_customers,
       SUM(CASE WHEN Exited = 1 THEN 1 ELSE 0 END) * 100.0 / COUNT(*) AS churn_rate
FROM churn_data
GROUP BY Balance
ORDER BY Balance
LIMIT 10
"""
balance_analysis = run_query(balance_query)
print(balance_analysis)


    Balance  total_customers  churned_customers  churn_rate
0      0.00             3617                500   13.823611
1   3768.69                1                  1  100.000000
2  12459.19                1                  1  100.000000
3  14262.80                1                  0    0.000000
4  16893.59                1                  0    0.000000
5  23503.31                1                  1  100.000000
6  24043.45                1                  1  100.000000
7  27288.43                1                  0    0.000000
8  27517.15                1                  0    0.000000
9  27755.97                1                  1  100.000000


In [43]:
num_products_query = """
SELECT NumOfProducts, 
       COUNT(*) AS total_customers, 
       SUM(CASE WHEN Exited = 1 THEN 1 ELSE 0 END) AS churned_customers,
       SUM(CASE WHEN Exited = 1 THEN 1 ELSE 0 END) * 100.0 / COUNT(*) AS churn_rate
FROM churn_data
GROUP BY NumOfProducts
ORDER BY NumOfProducts
"""
products_analysis = run_query(num_products_query)
print(products_analysis)


   NumOfProducts  total_customers  churned_customers  churn_rate
0              1             5084               1409   27.714398
1              2             4590                348    7.581699
2              3              266                220   82.706767
3              4               60                 60  100.000000


In [45]:
active_member_query = """
SELECT IsActiveMember, 
       COUNT(*) AS total_customers, 
       SUM(CASE WHEN Exited = 1 THEN 1 ELSE 0 END) AS churned_customers,
       SUM(CASE WHEN Exited = 1 THEN 1 ELSE 0 END) * 100.0 / COUNT(*) AS churn_rate
FROM churn_data
GROUP BY IsActiveMember
"""
active_member_analysis = run_query(active_member_query)
print(active_member_analysis)


   IsActiveMember  total_customers  churned_customers  churn_rate
0               0             4849               1302   26.850897
1               1             5151                735   14.269074


In [47]:
gender_churn_query = """
SELECT Gender, 
       COUNT(*) AS total_customers, 
       SUM(CASE WHEN Exited = 1 THEN 1 ELSE 0 END) AS churned_customers,
       SUM(CASE WHEN Exited = 1 THEN 1 ELSE 0 END) * 100.0 / COUNT(*) AS churn_rate
FROM churn_data
GROUP BY Gender
"""
gender_churn_analysis = run_query(gender_churn_query)
print(gender_churn_analysis)


   Gender  total_customers  churned_customers  churn_rate
0  Female             4543               1139   25.071539
1    Male             5457                898   16.455928
